# Example of using subpackage _siriuspy.servconf_

In [1]:
import siriuspy.servconf.conf_types as ct_module
import siriuspy.servconf.conf_service as cs_module
from pprint import pprint
import datetime
import time

## Configuration Types package

Print names of all defined configuration types for the service:

In [2]:
types = ct_module.get_config_types()
print(types)

('bo_strength_pvs', 'bo_tunes', 'si_strength_pvs', 'si_tunes', 'test')


get configuration value of a specific configuration type and prints its structure-defining dictionary

In [3]:
test_type_value = ct_module.get_config_type_value('test')
print(test_type_value)

{'paramA': 0.0, 'paramB': 0.0}


the module also implements a function that allows for checking whether a given config value is consistent with a configuration type:

In [4]:
arbitrary_value_dict = {'parameter_a':'value_a', 'parameter_b':'value_b'}
r = ct_module.check_value('test', arbitrary_value_dict)
print('{0:<22s} is of type "test" -> {1}'.format('"arbitrary_value_dict"', r))
r = ct_module.check_value('test', test_type_value)
print('{0:<22s} is of type "test" -> {1}'.format('"test_type_value"', r))

"arbitrary_value_dict" is of type "test" -> False
"test_type_value"      is of type "test" -> True


## Configuration Service packages

A connector object of class ConfigService is used to access the configuration database. An optional URL can be passed as argument. If not passed, the constructor will take the server address from siriusp.envars:

In [5]:
conn = cs_module.ConfigService(url=None) 

Conveniently the list of all configuration types can also be accessed through the connector:

In [6]:
conn.get_config_types()

('bo_strength_pvs', 'bo_tunes', 'si_strength_pvs', 'si_tunes', 'test')

### ConfigService.insert_config 

To insert a configuration of a particular type one can issue:

In [7]:
test = ct_module.get_config_type_value('test')
test['paramA'] = 4.27
response = conn.insert_config(config_type='test', name='ConfigName_Test1', value=test)
response = conn.insert_config(config_type='test', name='ConfigName_Test2', value=test)

The response object is a dictionary with keys that tells the result of the query:

In [8]:
print('Response code    : {}'.format(response['code']))
print('Response message : {}'.format(response['message']))

Response code    : 200
Response message : ok


If the query is successful there will be a _response_ key with the query data:

In [9]:
if 'result' in response:
    result = response['result']
    print('Configuration type      : {}'.format(result['config_type']))
    print('Configuration name      : {}'.format(result['name']))
    print('Configuration timestamp : {}'.format(result['created']))

Configuration type      : test
Configuration name      : ConfigName_Test2
Configuration timestamp : Tue, 24 Oct 2017 18:49:03 GMT


The _value_ of the inserted configuration is retrieved using the _value_ key

In [10]:
value = result['value']
print(value)

{'paramA': 4.27, 'paramB': 0.0}


### ConfigService.get_config

To retrieve a configuration given the config_type and the config name, get_config can be used

In [11]:
response = conn.get_config(config_type='test', name='ConfigName_Test1')
print("HTTP status code : {}".format(response['code']))
print("Message          : {}".format(response['message']))

HTTP status code : 200
Message          : ok


The fetched object can be accessed under the 'result' key:

In [12]:
if 'result' in response:
    config = response['result']
    pprint(config)

{'_id': '59ef8b1f8b4beb000c939eaf',
 'config_type': 'test',
 'created': 'Tue, 24 Oct 2017 18:49:03 GMT',
 'discarded': False,
 'modified': [],
 'name': 'ConfigName_Test1',
 'value': {'paramA': 4.27, 'paramB': 0.0}}


### ConfigService.update_config

To update an object in the database the update_config method can be used. Once you have the object you can change its field and pass it to the update method

In [13]:
config

{'_id': '59ef8b1f8b4beb000c939eaf',
 'config_type': 'test',
 'created': 'Tue, 24 Oct 2017 18:49:03 GMT',
 'discarded': False,
 'modified': [],
 'name': 'ConfigName_Test1',
 'value': {'paramA': 4.27, 'paramB': 0.0}}

In [14]:
config['value'] = {'paramA': 4.27, 'paramB': 15.0}
conn.update_config(config)

{'code': 200, 'message': 'ok', 'result': 1}

The `result` field in the response gives the number of objects modified. In this case only the object we passed as parameter was modified. If we query this object again, we can see the `value` field was updated

In [15]:
response = conn.get_config('test', 'ConfigName_Test1')
if 'result' in response:
    config = response['result']
    pprint(config)
else:
    print(response)

{'_id': '59ef8b1f8b4beb000c939eaf',
 'config_type': 'test',
 'created': 'Tue, 24 Oct 2017 18:49:03 GMT',
 'discarded': False,
 'modified': ['Tue, 24 Oct 2017 18:49:04 GMT'],
 'name': 'ConfigName_Test1',
 'value': {'paramA': 4.27, 'paramB': 15.0}}


Not all fields can be updated. The update method will ignore the `_id`, `config_type` and `timestamp` fields

### ConfigService.delete_config

The `delete_config` method is use to delete a configuration. Note, however, that a configuration is not truly deleted. This method simply sets the `deletable` field to true.

In [16]:
conn.delete_config(config)

{'code': 200, 'message': 'ok', 'result': 1}

The `result` field in the response gives the number of objects modified. In this case only the object we passed as parameter was modified. If we query this object again, we can see the `deletable` field was set to True as expected

In [17]:
response = conn.get_config('test', 'ConfigName_Test1')
if 'result' in response:
    config = response['result']
    pprint(config)

{'_id': '59ef8b1f8b4beb000c939eaf',
 'config_type': 'test',
 'created': 'Tue, 24 Oct 2017 18:49:03 GMT',
 'discarded': True,
 'modified': ['Tue, 24 Oct 2017 18:49:04 GMT', 'Tue, 24 Oct 2017 18:49:04 GMT'],
 'name': 'ConfigName_Test1',
 'value': {'paramA': 4.27, 'paramB': 15.0}}


### ConfigService.find_config 

The `find_config` method is used to get a list of objects that match a set of parameters like `name`, `timestamps` and wether the object is set as `deletable`.

In the command below we aks for all configurations of type 'test'. As a rule `find_config` won't return configurations that are set as `deletable`, so we set `deleted` to None so all configurations are returned.

In [18]:
response = conn.find_configs(config_type='test', discarded=None)
if 'result' in response:
    print("Configurations found: ", len(response['result']))
pprint(response)

Configurations found:  2
{'code': 200,
 'message': 'ok',
 'result': [{'_id': '59ef8b1f8b4beb000c939eaf',
             'config_type': 'test',
             'name': 'ConfigName_Test1'},
            {'_id': '59ef8b1f8b4beb000db661ed',
             'config_type': 'test',
             'name': 'ConfigName_Test2'}]}


The `begin` and `end` parameters accept a timestamp. They can be used to query the configurations on their creation timestamp. In the examples below the current timestamp is used. The first query, in which the `start` parameter is supplied, returns an empty list as there is no configuration created after the timestamp. In the second query we set the `end` parameter. The query returns a list with all the configurations created before the timestamp.

In [19]:
timestamp = time.time()  # Current timestamp
print("Current time: ", datetime.datetime.utcfromtimestamp(timestamp))
response = conn.find_configs(begin=timestamp, discarded=None)
pprint(response)

Current time:  2017-10-24 18:49:04.491322
{'code': 200, 'message': 'ok', 'result': []}


In [20]:
timestamp = time.time()  # Current timestamp
print("Current time: ", datetime.datetime.utcfromtimestamp(timestamp))
response = conn.find_configs(end=timestamp, discarded=None)
pprint(response)

Current time:  2017-10-24 18:49:04.534606
{'code': 200,
 'message': 'ok',
 'result': [{'_id': '59ef8b1f8b4beb000c939eaf',
             'config_type': 'test',
             'name': 'ConfigName_Test1'},
            {'_id': '59ef8b1f8b4beb000db661ed',
             'config_type': 'test',
             'name': 'ConfigName_Test2'}]}


### ConfigService.conv_timestamp

The utility function `conv_timestamp` can be used to convert the date string in an object's `timestamp` field, to an unix time stamp. It is useful when you want to create a custom date string

In [21]:
response = conn.get_config(config_type='test', name='ConfigName_Test1')
try:
    config = response['result']
except Exception:
    pprint(response)
t = conn.conv_timestamp(config['created'])
date = datetime.datetime.fromtimestamp(t)  # Get a datetime object with local time
print(date.strftime("%d/%m/%Y %H:%M:%S"))
date = datetime.datetime.utcfromtimestamp(t)  # Get a datetime object with utc time
print(date.strftime("%d/%m/%Y %H:%M:%S"))

24/10/2017 16:49:03
24/10/2017 18:49:03


### ConfigService.find_nr_configs

The `find_nr_configs` methods can be used if all you need is the number of document that match your searching criteria

In [22]:
response = conn.find_nr_configs(config_type="test", discarded=None) 
try:
    print("Configurations found:", response['result'])
except KeyError:
    print(response)

Configurations found: 2
